# Computación en matrices NumPy: funciones universales

Hasta ahora, hemos estado discutiendo algunos de los aspectos básicos de NumPy. En los próximos capítulos, profundizaremos en las razones por las que NumPy es tan importante en el mundo de la ciencia de datos de Python: concretamente, porque proporciona una interfaz sencilla y flexible para optimizar el cálculo con matrices de datos.

El cálculo en matrices NumPy puede ser muy rápido o muy lento.
La clave para hacerlo rápido es utilizar operaciones vectorizadas, generalmente implementadas a través de las *funciones universales* (ufuncs) de NumPy.
Este capítulo motiva la necesidad de las ufuncs de NumPy, que pueden usarse para hacer que los cálculos repetidos en elementos de una matriz sean mucho más eficientes.
Luego presenta muchas de las funciones aritméticas más comunes y útiles disponibles en el paquete NumPy.

## La lentitud de los bucles

La implementación predeterminada de Python (conocida como CPython) realiza algunas operaciones muy lentamente.
Esto se debe en parte a la naturaleza dinámica e interpretada del lenguaje; Los tipos son flexibles, por lo que las secuencias de operaciones no se pueden compilar en código de máquina eficiente como en lenguajes como C y Fortran.
Recientemente ha habido varios intentos de abordar esta debilidad: ejemplos bien conocidos son el [proyecto PyPy](http://pypy.org/), una implementación compilada de Python justo a tiempo; el [proyecto Cython](http://cython.org), que convierte el código Python en código C compilable; y el [proyecto Numba] (http://numba.pydata.org/), que convierte fragmentos de código Python en código de bytes LLVM rápido.
Cada uno de estos tiene sus fortalezas y debilidades, pero es seguro decir que ninguno de los tres enfoques ha superado aún el alcance y la popularidad del motor CPython estándar.

La relativa lentitud de Python generalmente se manifiesta en situaciones en las que se repiten muchas operaciones pequeñas; por ejemplo, recorrer matrices para operar en cada elemento.
Por ejemplo, imagina que tenemos una serie de valores y nos gustaría calcular el recíproco de cada uno.
Un enfoque sencillo podría verse así:

In [1]:
import numpy as np
rng = np.random.default_rng(seed=1701)

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = rng.integers(1, 10, size=5)
compute_reciprocals(values)

array([0.11111111, 0.25      , 1.        , 0.33333333, 0.125     ])

Esta implementación probablemente le parezca bastante natural a alguien con experiencia en C o Java, por ejemplo.
Pero si medimos el tiempo de ejecución de este código para una entrada grande, vemos que esta operación es muy lenta, ¡tal vez sorprendentemente!
Compararemos esto con la magia `%timeit` de IPython (que se analiza en [Código de creación de perfiles y sincronización] (01.07-Timing-and-Profiling.ipynb)):

In [2]:
big_array = rng.integers(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)

2.61 s ± 192 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


¡Se necesitan varios segundos para calcular estos millones de operaciones y almacenar el resultado!
Cuando incluso los teléfonos móviles tienen velocidades de procesamiento medidas en gigaflops (es decir, miles de millones de operaciones numéricas por segundo), esto parece casi absurdamente lento.
Resulta que el cuello de botella aquí no son las operaciones en sí, sino la verificación de tipos y los envíos de funciones que CPython debe realizar en cada ciclo del ciclo.
Cada vez que se calcula el recíproco, Python primero examina el tipo de objeto y realiza una búsqueda dinámica de la función correcta a utilizar para ese tipo.
Si, en cambio, estuviéramos trabajando en código compilado, esta especificación de tipo se conocería antes de que se ejecutara el código y el resultado podría calcularse de manera mucho más eficiente.

## Presentamos Ufuncs

Para muchos tipos de operaciones, NumPy proporciona una interfaz conveniente para este tipo de rutina compilada y escrita estáticamente. Esto se conoce como operación *vectorizada*.
Para operaciones simples como la división por elementos aquí, la vectorización es tan simple como usar operadores aritméticos de Python directamente en el objeto de matriz.
Este enfoque vectorizado está diseñado para empujar el bucle hacia la capa compilada que subyace a NumPy, lo que lleva a una ejecución mucho más rápida.

Compare los resultados de las dos operaciones siguientes:

In [3]:
print(compute_reciprocals(values))
print(1.0 / values)

[0.11111111 0.25       1.         0.33333333 0.125     ]
[0.11111111 0.25       1.         0.33333333 0.125     ]


Al observar el tiempo de ejecución de nuestra gran matriz, vemos que completa órdenes de magnitud más rápido que el bucle de Python:

In [4]:
%timeit (1.0 / big_array)

2.54 ms ± 383 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Las operaciones vectorizadas en NumPy se implementan a través de ufuncs, cuyo objetivo principal es ejecutar rápidamente operaciones repetidas sobre valores en matrices NumPy.
Las Ufuncs son extremadamente flexibles; antes vimos una operación entre un escalar y una matriz, pero también podemos operar entre dos matrices:

In [5]:
np.arange(5) / np.arange(1, 6)

array([0.        , 0.5       , 0.66666667, 0.75      , 0.8       ])

Y las operaciones ufunc no se limitan a matrices unidimensionales. También pueden actuar sobre matrices multidimensionales:

In [6]:
x = np.arange(9).reshape((3, 3))
2 ** x

array([[  1,   2,   4],
       [  8,  16,  32],
       [ 64, 128, 256]])

Los cálculos que utilizan la vectorización a través de ufuncs son casi siempre más eficientes que sus contrapartes implementadas usando bucles de Python, especialmente a medida que las matrices crecen en tamaño.
Cada vez que vea un bucle de este tipo en un script NumPy, debe considerar si se puede reemplazar con una expresión vectorizada.

## Explorando las funciones de NumPy

Las ufuncs existen en dos tipos: *ufuncs unarias*, que operan en una sola entrada, y *ufuncs binarias*, que operan en dos entradas.
Veremos ejemplos de ambos tipos de funciones aquí.

### Aritmética de matrices

Las ufuncs de NumPy resultan muy naturales de usar porque utilizan los operadores aritméticos nativos de Python.
Se pueden utilizar la suma, resta, multiplicación y división estándar:

In [7]:
x = np.arange(4)
print("x      =", x)
print("x + 5  =", x + 5)
print("x - 5  =", x - 5)
print("x * 2  =", x * 2)
print("x / 2  =", x / 2)
print("x // 2 =", x // 2)  # floor division

x      = [0 1 2 3]
x + 5  = [5 6 7 8]
x - 5  = [-5 -4 -3 -2]
x * 2  = [0 2 4 6]
x / 2  = [0.  0.5 1.  1.5]
x // 2 = [0 0 1 1]


También hay una ufunc unaria para negación, un operador `**` para exponenciación y un operador `%` para módulo:

In [8]:
print("-x     = ", -x)
print("x ** 2 = ", x ** 2)
print("x % 2  = ", x % 2)

-x     =  [ 0 -1 -2 -3]
x ** 2 =  [0 1 4 9]
x % 2  =  [0 1 0 1]


Además, se pueden unir como se desee y se respeta el orden estándar de operaciones:

In [9]:
-(0.5*x + 1) ** 2

array([-1.  , -2.25, -4.  , -6.25])

Todas estas operaciones aritméticas son simplemente envoltorios convenientes para ufuncs específicas integradas en NumPy. Por ejemplo, el operador `+` es un contenedor para la ufunc `add`:

In [10]:
np.add(x, 2)

array([2, 3, 4, 5])

La siguiente tabla enumera los operadores aritméticos implementados en NumPy:

| Operador | Ufunc equivalente | Descripción |
|-------------|-------------------|--------------- ----------------------|
|`+` |`np.add` |Suma (por ejemplo, `1 + 1 = 2`) |
|`-` |`np.subtract` |Resta (por ejemplo, `3 - 2 = 1`) |
|`-` |`np.negative` |Negación unaria (p. ej., `-2`) |
|`*` |`np.multiply` |Multiplicación (por ejemplo, `2 * 3 = 6`) |
|`/` |`np.divide` |División (p. ej., `3/2 = 1,5`) |
|`//` |`np.floor_divide` |División del piso (por ejemplo, `3 // 2 = 1`) |
|`**` |`np.power` |Exponciación (p. ej., `2 ** 3 = 8`) |
|`%` |`np.mod` |Módulo/resto (por ejemplo, `9 % 4 = 1`)|

Además, existen operadores booleanos/bit a bit; los exploraremos en [Comparaciones, máscaras y lógica booleana] (02.06-Boolean-Arrays-and-Masks.ipynb).

### Valor absoluto

Así como NumPy comprende los operadores aritméticos integrados de Python, también comprende la función de valor absoluto integrada de Python:

In [11]:
x = np.array([-2, -1, 0, 1, 2])
abs(x)

array([2, 1, 0, 1, 2])

La ufunc NumPy correspondiente es `np.absolute`, que también está disponible bajo el alias `np.abs`:

In [12]:
np.absolute(x)

array([2, 1, 0, 1, 2])

In [13]:
np.abs(x)

array([2, 1, 0, 1, 2])

Este ufunc también puede manejar datos complejos, en cuyo caso devuelve la magnitud:

In [14]:
x = np.array([3 - 4j, 4 - 3j, 2 + 0j, 0 + 1j])
np.abs(x)

array([5., 5., 2., 1.])

### Funciones trigonométricas

NumPy proporciona una gran cantidad de funciones útiles, y algunas de las más útiles para el científico de datos son las funciones trigonométricas.
Comenzaremos definiendo una serie de ángulos:

In [15]:
theta = np.linspace(0, np.pi, 3)

Ahora podemos calcular algunas funciones trigonométricas con estos valores:

In [16]:
print("theta      = ", theta)
print("sin(theta) = ", np.sin(theta))
print("cos(theta) = ", np.cos(theta))
print("tan(theta) = ", np.tan(theta))

theta      =  [0.         1.57079633 3.14159265]
sin(theta) =  [0.0000000e+00 1.0000000e+00 1.2246468e-16]
cos(theta) =  [ 1.000000e+00  6.123234e-17 -1.000000e+00]
tan(theta) =  [ 0.00000000e+00  1.63312394e+16 -1.22464680e-16]


Los valores se calculan dentro de la precisión de la máquina, razón por la cual los valores que deberían ser cero no siempre llegan exactamente a cero.
También están disponibles funciones trigonométricas inversas:

In [17]:
x = [-1, 0, 1]
print("x         = ", x)
print("arcsin(x) = ", np.arcsin(x))
print("arccos(x) = ", np.arccos(x))
print("arctan(x) = ", np.arctan(x))

x         =  [-1, 0, 1]
arcsin(x) =  [-1.57079633  0.          1.57079633]
arccos(x) =  [3.14159265 1.57079633 0.        ]
arctan(x) =  [-0.78539816  0.          0.78539816]


### Exponentes y logaritmos

Otras operaciones comunes disponibles en NumPy ufuncs son las exponenciales:

In [18]:
x = [1, 2, 3]
print("x   =", x)
print("e^x =", np.exp(x))
print("2^x =", np.exp2(x))
print("3^x =", np.power(3., x))

x   = [1, 2, 3]
e^x = [ 2.71828183  7.3890561  20.08553692]
2^x = [2. 4. 8.]
3^x = [ 3.  9. 27.]


También están disponibles los inversos de las exponenciales, los logaritmos.
El `np.log` básico proporciona el logaritmo natural; Si prefiere calcular el logaritmo de base 2 o el logaritmo de base 10, también están disponibles:

In [19]:
x = [1, 2, 4, 10]
print("x        =", x)
print("ln(x)    =", np.log(x))
print("log2(x)  =", np.log2(x))
print("log10(x) =", np.log10(x))

x        = [1, 2, 4, 10]
ln(x)    = [0.         0.69314718 1.38629436 2.30258509]
log2(x)  = [0.         1.         2.         3.32192809]
log10(x) = [0.         0.30103    0.60205999 1.        ]


También existen algunas versiones especializadas que son útiles para mantener la precisión con entradas muy pequeñas:

In [20]:
x = [0, 0.001, 0.01, 0.1]
print("exp(x) - 1 =", np.expm1(x))
print("log(1 + x) =", np.log1p(x))

exp(x) - 1 = [0.         0.0010005  0.01005017 0.10517092]
log(1 + x) = [0.         0.0009995  0.00995033 0.09531018]


Cuando `x` es muy pequeño, estas funciones dan valores más precisos que si se usaran `np.log` o `np.exp` sin formato.

### Funciones especializadas

NumPy tiene muchas más funciones disponibles, incluidas trigonometría hiperbólica, aritmética bit a bit, operaciones de comparación, conversiones de radianes a grados, redondeo y residuos, y mucho más.
Una mirada a la documentación de NumPy revela muchas funciones interesantes.

Otra excelente fuente de ufuncs más especializados es el submódulo `scipy.special`.
Si desea calcular alguna función matemática oscura con sus datos, es probable que esté implementada en `scipy.special`.
Hay demasiadas funciones para enumerarlas todas, pero el siguiente fragmento muestra un par que podrían surgir en un contexto estadístico:

In [21]:
from scipy import special

In [22]:
# Gamma functions (generalized factorials) and related functions
x = [1, 5, 10]
print("gamma(x)     =", special.gamma(x))
print("ln|gamma(x)| =", special.gammaln(x))
print("beta(x, 2)   =", special.beta(x, 2))

gamma(x)     = [1.0000e+00 2.4000e+01 3.6288e+05]
ln|gamma(x)| = [ 0.          3.17805383 12.80182748]
beta(x, 2)   = [0.5        0.03333333 0.00909091]


In [23]:
# Error function (integral of Gaussian),
# its complement, and its inverse
x = np.array([0, 0.3, 0.7, 1.0])
print("erf(x)  =", special.erf(x))
print("erfc(x) =", special.erfc(x))
print("erfinv(x) =", special.erfinv(x))

erf(x)  = [0.         0.32862676 0.67780119 0.84270079]
erfc(x) = [1.         0.67137324 0.32219881 0.15729921]
erfinv(x) = [0.         0.27246271 0.73286908        inf]


Hay muchas, muchas más funciones disponibles tanto en NumPy como en `scipy.special`.
Debido a que la documentación de estos paquetes está disponible en línea, una búsqueda en la web como "función gamma python" generalmente encontrará la información relevante.

## Funciones avanzadas de Ufunc

Muchos usuarios de NumPy utilizan ufuncs sin siquiera conocer su conjunto completo de funciones.
Aquí describiré algunas características especializadas de ufuncs.

### Especificación de salida

Para cálculos grandes, a veces resulta útil poder especificar la matriz donde se almacenará el resultado del cálculo.
Para todas las ufuncs, esto se puede hacer usando el argumento `out` de la función:

In [24]:
x = np.arange(5)
y = np.empty(5)
np.multiply(x, 10, out=y)
print(y)

[ 0. 10. 20. 30. 40.]


Esto incluso se puede utilizar con vistas de matriz. Por ejemplo, podemos escribir los resultados de un cálculo en cualquier otro elemento de una matriz especificada:

In [25]:
y = np.zeros(10)
np.power(2, x, out=y[::2])
print(y)

[ 1.  0.  2.  0.  4.  0.  8.  0. 16.  0.]


Si en lugar de eso hubiéramos escrito `y[::2] = 2 ** x`, esto habría resultado en la creación de una matriz temporal para contener los resultados de `2 ** x`, seguida de una segunda operación copiando esos valores. en la matriz `y`.
Esto no supone una gran diferencia para un cálculo tan pequeño, pero para matrices muy grandes el ahorro de memoria gracias al uso cuidadoso del argumento "out" puede ser significativo.

### Agregaciones

Para ufuncs binarios, las agregaciones se pueden calcular directamente desde el objeto.
Por ejemplo, si queremos *reducir* una matriz con una operación particular, podemos usar el método `reducir` de cualquier ufunc.
Una reducción aplica repetidamente una operación determinada a los elementos de una matriz hasta que solo queda un resultado.

Por ejemplo, llamar a `reduce` en la ufunc `add` devuelve la suma de todos los elementos de la matriz:

In [26]:
x = np.arange(1, 6)
np.add.reduce(x)

15

De manera similar, llamar a "reducir" en la ufunc "multiplicar" da como resultado el producto de todos los elementos de la matriz:

In [27]:
np.multiply.reduce(x)

120

Si queremos almacenar todos los resultados intermedios del cálculo, podemos usar `acumular`:

In [28]:
np.add.accumulate(x)

array([ 1,  3,  6, 10, 15])

In [29]:
np.multiply.accumulate(x)

array([  1,   2,   6,  24, 120])

Tenga en cuenta que para estos casos particulares, existen funciones NumPy dedicadas para calcular los resultados (`np.sum`, `np.prod`, `np.cumsum`, `np.cumprod`), que exploraremos en [Agregaciones : Mínimo, máximo y todo lo intermedio] (02.04-Computación-en-arrays-aggregates.ipynb).

### Productos externos

Finalmente, cualquier ufunc puede calcular la salida de todos los pares de dos entradas diferentes usando el método "externo".
Esto le permite, en una línea, hacer cosas como crear una tabla de multiplicar:

In [30]:
x = np.arange(1, 6)
np.multiply.outer(x, x)

array([[ 1,  2,  3,  4,  5],
       [ 2,  4,  6,  8, 10],
       [ 3,  6,  9, 12, 15],
       [ 4,  8, 12, 16, 20],
       [ 5, 10, 15, 20, 25]])

Los métodos `ufunc.at` y `ufunc.reduceat` también son útiles y los exploraremos en [Fancy Indexing](02.07-Fancy-Indexing.ipynb).

También encontraremos la capacidad de ufuncs para operar entre matrices de diferentes formas y tamaños, un conjunto de operaciones conocidas como *difusión*.
Este tema es lo suficientemente importante como para dedicarle un capítulo completo (consulte [Computación en matrices: transmisión] (02.05-Computación-en-arrays-broadcasting.ipynb)).

## Ufuncs: aprender más

Puede encontrar más información sobre funciones universales (incluida la lista completa de funciones disponibles) en los sitios web de documentación [NumPy](http://www.numpy.org) y [SciPy](http://www.scipy.org) .

Recuerde que también puede acceder a la información directamente desde IPython importando los paquetes y utilizando la funcionalidad de ayuda y finalización de pestañas de IPython (`?`), como se describe en [Ayuda y documentación en IPython] (01.01-Ayuda-y-Documentación.ipynb) .